In [ ]:
# version mise a jour en juin 2024 pour une nouvelle structure de fichier .pkl
# et pour travailler avec des dataframe

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import json
import time
from matplotlib.backend_bases import MouseButton
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd
import pickle
import datetime
import os
import matplotlib as mpl
import scipy
import scipy.stats
import nitime
from scipy.interpolate import RegularGridInterpolator
from pathlib import Path
import netCDF4

#from statistics import linear_regression
from scipy.optimize import curve_fit

# Utilities

In [ ]:
def gety_OLD(depth,signal,xp):
    #deprecated
    ind = [np.argwhere(depth == xp[i])[0][0] for i in range(len(xp))]
    return list(signal[ind])

def gety(depth,signal,xp):
    return np.interp(xp,depth,signal)

def getlinks(ax,xp1,xp2,yp1,yp2):
    
    combinedTransform = ax[1].transData + ax[0].transData.inverted()

    
    xyp1 = [combinedTransform.inverted().transform(bob) for bob in zip(xp1,yp1)]
    xp1_on_ax2 = [bob[0] for bob in xyp1]
    yp1_on_ax2 = [bob[1] for bob in xyp1]
        
    xp_links = np.full(len(xp1)+2*len(xp2),np.nan)
    yp_links = xp_links.copy()
    
    xp_links[::3] = xp1_on_ax2.copy()
    xp_links[1::3] = xp2.copy()
    
    yp_links[::3] = yp1_on_ax2.copy()
    yp_links[1::3] = yp2.copy()

    return xp_links,yp_links

# basic file handling

In [ ]:
def load_dic_file(filename):
    
    with open(filename, 'rb') as fp:
        dic = pickle.load(fp)
        
    return dic

def write_dic_file(dic,filename):
    
    with open(filename, 'wb') as fp:
        pickle.dump(dic, fp)      

In [1]:
def load_marked_points(aligfile):
        # convert the tiepoints stored in the dictionary to the xp1 and xp2 lists
        
        new_dic = load_dic_file(aligfile)
        
        xp1_dic = {}
        xp2_dic = {}
        
        for profile_key in new_dic['tiepoints'].keys():
        
            tiepoints = new_dic['tiepoints'][profile_key]
            xp1 = [bob['profile_depth'] for bob in tiepoints]
            xp2 = [bob['ref_depth'] for bob in tiepoints]
        
            if len(xp2)>0:
                # sort points
                xp1,xp2 = zip(*sorted(zip(xp1,xp2)))

            xp1_dic[profile_key] = xp1
            xp2_dic[profile_key] = xp2
        
        return xp1_dic,xp2_dic

In [ ]:
def load_marked_points_old(aligfile):
    
    # load depth dictionary from alignment file
    
    new_dic = load_dic_file(aligfile)
    xp1_dic = {}
    xp2_dic = {}
    
    for profile_key in new_dic['tiepoints'].keys():
        xp1 = new_dic['tiepoints'][profile_key]['xp1']
        xp2 = new_dic['tiepoints'][profile_key]['xp2']
        
        if len(xp2)>0:
            # sort points
            xp1,xp2 = zip(*sorted(zip(xp1,xp2)))

        xp1_dic[profile_key] = xp1
        xp2_dic[profile_key] = xp2
    
                
    return xp1_dic,xp2_dic

In [ ]:
def load_profiles_data(aligfile):
    
    full_dic = load_dic_file(aligfile)
    
    cores_dic = {key:value for key,value in full_dic['cores'].items() if key !='REF'}
    ref_dic = full_dic['cores']['REF']
    
    return ref_dic,cores_dic

## trench setup


In [ ]:
def common_depth_scale(depthdictionary,dz):
    
    dz = 3
    zmin = min([min(value) for key,value in depthdictionary.items()])
    zmax = max([max(value) for key,value in depthdictionary.items()])
    depth_scale = np.arange(zmin,zmax,dz)
    
    return depth_scale

### exploiting alignment

In [ ]:
# to convert an xlsx file to a dictionary

def load_xlsdata(datafile):
    xls = pd.ExcelFile(datafile)

    # each sheet corresponds to a pit

    core_names = xls.sheet_names[:]
    
    new_dic ={}
    
    for i,lab in enumerate(core_names):
    
        df = pd.read_excel(datafile,sheet_name=i,skiprows = 0, header = None).replace('n.a.', np.nan)
    
        #sample_date_timestamp = df.iloc[0,3]
        #sample_date = sample_date_timestamp.date()
        sample_date = datetime.date(2000,1,1)
    
        core_depth = df.to_numpy()[1:,0].astype(None)
    
        #print(core_depth)
    
        new_dic[lab] = {}
        
        for i in range(1,len(df.T)):
            chem_name = df.to_numpy()[0,i]
            chem_profile = df.to_numpy()[1:,i].astype(None)
            new_dic[lab][chem_name] = {}

            new_dic[lab][chem_name]['depth'] = core_depth.copy()

            new_dic[lab][chem_name]['data'] = chem_profile.copy()
            
    return new_dic

In [ ]:
def depth_sqz_str(depth1,xp1,xp2):
    # JUNE2024: we still need to improve this function for coinciding xp1 points (hiatus)
    # roughly the same as np.interp but no error if xp1 is empty
    depth_out = np.full(depth1.shape,np.nan)
    if len(xp2)>0:
        # sort points
        xp1,xp2 = zip(*sorted(zip(xp1,xp2)))
        depth_out = np.interp(depth1,xp1[:len(xp2)],xp2)
    return depth_out

### if we want to visualize the stretch applied on each profile
deprecated: think about what we need now for depth profiles



def load_stretch_array(aligfile,vertical_scale = None,labels=None):
    
    # create an array with 
    
    ref_dic,cores_dic = load_profiles_data(aligfile)
    
    xp1_dic,xp2_dic = load_marked_points(aligfile)
    
    if labels is None: 
        labels = cores_dic.keys()
    
    if vertical_scale is None:
        return 'Please specify vertical scale!'
        #vertical_scale = ref_dic['depth'].copy()
        
        
    npits = len(labels)
    
    #data = chem_profiles[species]
    
    out = {}
    
    for i,pitlab in enumerate(labels):
        if pitlab in cores_dic.keys():
            
            depth_new = depth_sqz_str(cores_dic[pitlab]['depth'],xp1_dic[pitlab],xp2_dic[pitlab])

            gradient = np.diff(depth_new)/np.diff(cores_dic[pitlab]['depth'])
    
            gradient_out = np.interp(vertical_scale,depth_new[:-1],gradient,left=np.nan,right=np.nan)
        
            out[:,i] = gradient_out

                
    df = pd.DataFrame(data=out)
    df.index = vertical_scale.copy()
    
    return df

In [2]:
## TODO JUNE 2024: merge the following into a single function with a master function
## wrapper1: from the alignment file, specifying a species
## wrapper2: from the alignment file and the datafile, specifying a species

In [ ]:
def load_alig_array(aligfile,species,vertical_scale= None,labels = None):
    
    # create an array with 
    
    ref_dic,cores_dic = load_profiles_data(aligfile)
    
    xp1_dic,xp2_dic = load_marked_points(aligfile)

    
    if vertical_scale is None:
        vertical_scale = ref_dic[species]['depth'].copy()
             
    if labels is None: 
        labels = cores_dic.keys()
        
    npits = len(labels)
    
    #data = chem_profiles[species]
    
    out = {}
        
    for profile in labels:
        if species in cores_dic[profile] and profile in xp1_dic.keys():
            
            depth_new = depth_sqz_str(cores_dic[profile][species]['depth'],xp1_dic[profile],xp2_dic[profile])
    
            signal_new = np.interp(vertical_scale,depth_new,cores_dic[profile][species]['data']
                                   ,left=np.nan,right=np.nan)
    
            out[profile] = signal_new.copy()
    
        else:
            print('missing data or tiepoints for ',profile)
            out[profile] = np.full(len(vertical_scale),np.nan)
            

    df = pd.DataFrame(data=out)
    df.index = vertical_scale.copy()
    
    return df

In [ ]:
def load_aligned_datafile(aligfile,datafile,species,vertical_scale= None,labels = None):
    
    ref_dic,cores_dic = load_profiles_data(aligfile)
    
    xp1_dic,xp2_dic = load_marked_points(aligfile)

    
    if vertical_scale is None:
        vertical_scale = ref_dic[species]['depth'].copy()
             
    if labels is None: 
        labels = cores_dic.keys()
        
    npits = len(labels)
    
    #data = chem_profiles[species]
    
    datadic = load_xlsdata(datafile)
    
    depth_profiles2 = {key:datadic[key][species]['depth'] for key in datadic.keys()}
    data2 = {key:datadic[key][species]['data'] for key in datadic.keys()}
    
    out = {}
        
    for profile in labels:
        if profile in depth_profiles2.keys() and profile in xp1_dic.keys():
            
            depth_new = depth_sqz_str(depth_profiles2[profile],xp1_dic[profile],xp2_dic[profile])

    
            signal_new = np.interp(vertical_scale,depth_new,data2[profile],left=np.nan,right=np.nan)
    
            out[profile] = signal_new.copy()
    
        else:
            out[:,i] = np.full(len(vertical_scale),np.nan)
            

    df = pd.DataFrame(data=out)
    df.index = vertical_scale.copy()
    
    return df

### when we want to load data on a common vertical scale without any alignment

In [ ]:
def load_interp_datafile(datafile,species,vertical_scale = None,labels = None):
    datadic = load_xlsdata(datafile)
    return load_interp_dic(datadic,species,vertical_scale,labels)  

In [ ]:
def load_interp_dic(datadic,species,vertical_scale = None,labels = None):  
    
    depth_profiles2 = {key:datadic[key][species]['depth'] for key in datadic.keys()}
    data = {key:datadic[key][species]['data'] for key in datadic.keys()}
    
             
    if labels is None: 
        labels = depth_profiles2.keys()
        
    if vertical_scale is None:
        vertical_scale = depth_profiles2[list(labels)[0]].copy()
    
    out = {}
        
    for profile in labels:         
    
        signal_new = np.interp(vertical_scale,depth_profiles2[profile],data[profile],left=np.nan,right=np.nan)
    
        out[profile] = signal_new.copy()            

    df = pd.DataFrame(data=out)
    df.index = vertical_scale.copy()
    
    return df

# power spectral densities

In [ ]:
def mtm_psd(signal_in,fs,detrend = True):
    
    signal = signal_in.copy()
    # linear detrending
    if detrend:
        xs = np.array(range(len(signal)))
        a,b = scipy.stats.linregress(xs, signal, alternative='two-sided')[:2]
        trend = a*xs+b
        signal -= trend
    
    ff,pxx = nitime.algorithms.multi_taper_psd(
    signal,NW = 2.5,adaptive=False, jackknife=False, Fs = fs, sides = 'onesided'
    )[:2]
    return ff,pxx

In [ ]:
def PSD_MSMn(data_int,fs):
    
    # takes an input a data array and returns the mean PSD, PSD of mean and PSD's of columns
    
    # data_int is assumed to be already equaly sampled AND filled
    
    # Mean of power spectrum
    # S power spectrum of mean
    # Mn = M/n Mean power spectrum divided by number of arrays
    
    npits = data_int.shape[1]
        
    datamean = mean_signal(data_int)
    
    freq0,pxx0 = mtm_psd(datamean,fs)
    
    PXX = np.full([freq0.size,npits],np.nan)
    for j in range(npits):
        PXX[:,j] = mtm_psd(data_int[:,j],fs)[1]
        
    PXXmean = np.mean(PXX,axis=1)
    
    return freq0,pxx0,PXXmean,PXX

In [ ]:
def get_SNR(pxx0,PXXmean,PXX):
    npits = PXX.shape[1]
    
    M = PXXmean
    S = pxx0

    C = S-M/npits
    N = (M-S)
    
    SNR = np.maximum(C/N,np.full(C.shape,0)) # reject negative values
    
    return C,N,SNR